In [6]:
from langchain_ollama import ChatOllama
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import os
from pinecone import Pinecone
from pinecone_plugins.assistant.models.chat import Message
API_KEY = "....."       




In [7]:
llm = ChatOllama(
            model="llama3.2",
            temperature=0,
            verbose=True
        )

In [8]:
@tool
def qna(question : str) -> str:
    """Returns the answer to the question using the assistant"""
    pc = Pinecone(api_key=API_KEY)
    assistant = pc.assistant.Assistant(assistant_name='newmultimodal')

    msg = Message(content=question)
    resp = assistant.chat(messages=[msg])
    return resp['message']['content']


# print(qna("newmultimodal", "Tell me about patient and heart beat in visit 4"))


tools = [qna]

In [9]:
prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "I will just make the qna call to get the answer to the question and improve the response, the output title should be Patient past visits information.", 
                ),
                ("user", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

In [10]:
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt    
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "Give patient details"})

print("&&&")
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `qna` with `{'question': 'patient details'}`


The patient, John Smith, is 78 years old. He has had multiple visits for various health issues, including routine check-ups, follow-ups for hypertension, diabetes management, and post-TIA (Transient Ischemic Attack) care. He has also been treated for CHF (Congestive Heart Failure) exacerbations.

During his visits, his vitals and lab results have been recorded as follows:

- **Visit 1 (Routine Check-Up on 2024-06-15)**:
  - Blood Pressure: 150/85 mmHg
  - Heart Rate: 68 bpm
  - Respiratory Rate: 18 breaths/min
  - Temperature: 98.1°F
  - Weight: 195 lbs
  - BMI: 29.0
  - HbA1c: 7.8%
  - Creatinine: 1.4 mg/dL
  - Sodium: 137 mEq/L
  - Potassium: 4.1 mEq/L
  - Cholesterol Total: 205 mg/dL
  - LDL: 130 mg/dL
  - Notes: Complaints of mild dizziness, elevated HbA1c, and elevated BP.

- **Visit 2 (Post-TIA Follow-Up on 2024-07-02)**:
  - Blood Pressure: 160/90 mmHg
  - Heart Rate: 72 bpm
  - Res